In [1]:
from datetime import datetime, timedelta
import pandas
import numpy
import statsmodels.api as sm
from statsmodels.tools import categorical, add_constant
from statsmodels.genmod.generalized_linear_model import GLM
import scipy.stats as sps
from IPython.display import display

## Dataset


In [2]:
display(pandas.read_csv('pydata_vw_tweets.csv').tail())

,created_at,text
152277,2014-06-01T06:37:09.000Z,A few improvements prior to #bristolvolksfest ...
152278,2014-06-01T06:31:47.000Z,VW to accelerate US model rollout - Filed unde...
152279,2014-06-01T03:19:06.000Z,#Volkswagen Group to invest 100 million euros ...
152280,2014-06-01T01:27:42.000Z,VW to accelerate US model rollout: Filed under...
152281,2014-06-01T01:08:41.000Z,The end. Once more time thank you so much @veu...


## Dataframe generation

In [3]:
def create_tweets_df(starting_date, final_date, time_window):
    """
    Create pandas DataFrame with tweets ranging between two dates and resample
    it to represent intervals of fixed duration.
    
    :param time_window: str e.g. '60Min', or '10Min'
    """

    # csv has timestamps in first column named 'created_at'
    tweets_df = pandas.DataFrame(pandas.read_csv(
        'pydata_vw_tweets.csv', parse_dates=[0]))
    tweets_df = tweets_df.created_at
    tweets_df = pandas.DataFrame(tweets_df)

    tweets_df.loc[:, 'n'] = 1
    
    tweets_df.set_index(['created_at'], inplace=True)

    # dummy tweet is used to resample until the very end
    dummy_tweet = {'created_at': final_date,
                   'n': 1}
    dummy_tweets_df = pandas.DataFrame([dummy_tweet]).set_index('created_at')

    aux_df = pandas.concat([dummy_tweets_df, tweets_df])
    
    # resample dataframe and count number of tweets for every time_window (0 if no tweet) 
    df_reshaped = aux_df.resample(time_window, label='left', closed='left').sum().fillna(0)

    df_reshaped.loc[:, 'weekday'] = df_reshaped.index.map(lambda x: x.weekday)
    df_reshaped.loc[:, 'hour'] = df_reshaped.index.map(lambda x: x.hour)
    # shift by 1 since it's n for previous period
    df_reshaped.loc[:, 'n_prev_period'] = df_reshaped.n.shift(1).fillna(0)
    df_reshaped.loc[:, 'n_prev_2_period'] = df_reshaped.n_prev_period.shift(1).fillna(0)

    df_reshaped = df_reshaped.ix[starting_date:final_date].ix[:-1]
    return df_reshaped

In [5]:
# load some data between two specified dates
data = create_tweets_df(datetime(2014, 6, 3, 16), datetime(2015, 10, 4),  '60Min')
print data.head()
print data.tail()

                        n  weekday  hour  n_prev_period  n_prev_2_period
created_at                                                              
2014-06-03 16:00:00  32.0        1    16           15.0              5.0
2014-06-03 17:00:00   7.0        1    17           32.0             15.0
2014-06-03 18:00:00   2.0        1    18            7.0             32.0
2014-06-03 19:00:00   3.0        1    19            2.0              7.0
2014-06-03 20:00:00   5.0        1    20            3.0              2.0
                        n  weekday  hour  n_prev_period  n_prev_2_period
created_at                                                              
2015-10-03 19:00:00  10.0        5    19           37.0              6.0
2015-10-03 20:00:00  18.0        5    20           10.0             37.0
2015-10-03 21:00:00  26.0        5    21           18.0             10.0
2015-10-03 22:00:00   9.0        5    22           26.0             18.0
2015-10-03 23:00:00   3.0        5    23           

## Model creation and fitting

In [6]:
def create_model(training_data):
    """
    Create a Poisson model given training data in a pandas DataFrame form

    :param training_data: pandas.DataFrame with specific columns for predictors
    :return:
    """

    model_family = sm.families.Poisson()
    
    # use categorical variable, which means that e.g. to represent day 4, the array is [0 0 0 0 1 0 0]
    dummy_hour = categorical(numpy.asarray(training_data['hour']))[:, 1:]
    dummy_weekday = categorical(numpy.asarray(training_data['weekday']))[:, 1:]

    # target variable
    endog = numpy.asmatrix(training_data['n']).T

    # combine predictors
    exog = numpy.concatenate((training_data[['n_prev_period', 'n_prev_2_period']],
                           dummy_hour, dummy_weekday), axis=1)
    exog = add_constant(exog, prepend=False)

    model = GLM(endog, exog, family=model_family)
    return model

In [13]:
def generate_and_fit_model(training_data):
    """
    Create and fit GLM model on training data
    
    :param: training_data: data to use when training the model
    :return: fitted model
    """
    
    glm_model = create_model(training_data)

    # fit the model
    model_results = glm_model.fit(maxiter=100)
    print "Model generated."

    return model_results

## Detection

In [8]:
def create_design_matrix(n_minus_1, n_minus_2, hour, weekday):
    """
    Create the design matrix for predictors for a single datapoint,
    each column correspond to a predictor.
    NB: hour and day are set as indicators,
        i.e. arrays with 0s and one element set to 1

    :param n_minus_1: number of tweets in the previous period
    :param n_minus_2: number of tweets seen 2 periods earlier
    :param hour: hour of the day
    :param weekday: index for the day of the week, Monday is 0
    :return: matrix with predictors
    """
    hour_indicator = [0] * 24
    hour_indicator[int(hour)] = 1

    day_indicator = [0] * 7
    day_indicator[int(weekday)] = 1
    
    # add constant which refer to the intercept 
    constant = 1
    return numpy.concatenate(([n_minus_1], [n_minus_2], hour_indicator,
                           day_indicator, [constant]))

In [9]:
# example of one single datapoint in the feature space
display(create_design_matrix(10, 8, 6, 2))

array([10,  8,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1])

In [10]:
def predict(model_results, design_matrix, n_events, alpha):
    """
    Detect if the volume is anomalous using a given model and threshold.

    :param model_results: fitted model
    :param design_matrix: datapoint in the predictors space
    :param n_events: volume of tweet in the current window - check if this is anomalous, given the model
    :param alpha: threshold 
    :return:
    """

    prediction = model_results.model.predict(
        model_results.params, design_matrix)
    # compute the number of events which correspond to probability threshold alpha. 
    # This represents the upper limit for the numeber of tweets created in the current period
    upper_limit = max(sps.poisson.ppf(1 - alpha, prediction), 0)
    is_anomaly = n_events > upper_limit
    
    # Similar to computing the probability of seeing more than n events 
    # and comparing directly with alpha.
    # Only difference is that upper_limit is a discrete value
    
    #     p_value = (1 - sps.poisson.cdf(k=n_events, mu=prediction))
    #    is_anomaly = p_value < alpha
    
    return bool(is_anomaly)

In [11]:
def detect(n_tweets, predictors, threshold_alpha, training_data):
    """
    Detect if the number of tweets is anomalous, given the predictors and a
    lower tail threshold

    :param n_tweets: number of tweets seen in the current period
    :param predictors: dictionary with predictors
    :param threshold_alpha: threshold for the pvalue
    :param training_data: dataframe with training data

    :return:
    """
    glm_model = generate_and_fit_model(training_data)

    n_minus_1 = predictors['n_prev_period']
    n_minus_2 = predictors['n_prev_2_period']
    weekday = predictors['weekday']
    hour = predictors['hour']

    design_matrix = create_design_matrix(n_minus_1, n_minus_2, hour, weekday)
    return predict(glm_model, design_matrix, n_tweets, threshold_alpha)

## Example - Single datapoint detection

In [19]:
predictors = {'n_prev_period': 3, 
              'n_prev_2_period': 5, 
              'hour': 16, 
              'weekday': 4}

example_data = create_tweets_df(datetime(2014, 5, 1), datetime(2016, 5, 5), time_window='10Min')
is_anomaly_example = detect(n_tweets=19, predictors=predictors, 
                            threshold_alpha=1e-4, training_data=example_data)
print "Anomalous? {0}".format(is_anomaly_example)

Model generated.
Anomalous? True


## Conclusions
* applicable to any data about events happening in time
* quick to train and fast in the detection
* can be used with real time data to detect/predict the next datapoint coming in (no need to wait for more data)
* easily extendible to include more and different predictors